Se tiene una base de datos de evaluación de automóviles, en donde cada automovil se ha caracterizado mediante 6 atributos:  

1. La precio de compra, llamado *buying* , que puede tener los siguientes valores: vhigh, high, med, low. 
2. El costo del mantenimiento, llamado *maint* que puede tener los siguientes valores: vhigh, high, med, low. 
3. El número de puertas, llamado *doors*, que puede tener los siguientes valores: 2, 3, 4, 5more. 
4. La capacidad experesada en el número de personas, llamado *persons*, que puede tener los siguientes valores: 2, 4, more. 
5. El tamaño del portamaletas,llamado *lug_boot*, que puede tener los siguientes valores: small, med, big. 
6. La seguridad estimada del auto, llamado *safety*,que puede tener los siguientes valores: low, med, high. 

De acuerdo con estas seis características, el auto puede ser clasificado en inaceptable (*unacc*), aceptable (*acc*), bueno (*good*) o muy bueno (*vgood*).

Le piden construir un sistema experto que clasifique automáticamente los autores utilizando la base de datos mencionada para su entrenamiento y validación, y el modelo de perceptrón como algoritmo de aprendizaje de máquina. Para la construcción del sistema tenga en cuenta las siguientes condiciones:

1. La base de datos y su descripción se encuentra el la página https://archive.ics.uci.edu/ml/datasets/Car+Evaluation. Debe descargar los datos y separarlos de las etiquetas de forma correcta. 
2. El perceptron no acepta variables categóricas como entradas o salidas (letras), por tanto debe convertir las variables que tiene a valores numéricos (variables numéricas). La forma en se haga la conversión es libre, perodebe ser especificada en los comentarios del código. Recuerde que debe hacer esto tanto para los datos como para las etiquetas.
3. Se debe tener una etapa de preprocesamiento.
4. Se debe mostrar el acierto de clasificación.
5. Si requiere alguna información que no está especificada en las condiciones, la debe suponer y justificar correctamente.

In [2]:
import numpy as np
import math
from numpy import genfromtxt
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn import neighbors, datasets
import neurolab as nl

#Cargamos los datos del archivo.
data = genfromtxt('car.data', dtype=str, delimiter=',')
print(data.shape)

(1728, 7)


In [3]:
def preprocesar(matriz):
    matriz_procesada = np.zeros_like(matriz, dtype=int) #Creamos la matriz donde guardaremos los datos
    filas = np.shape(matriz)[0] 
    columnas = np.shape(matriz)[1]
    valor3 = set(["vhigh","5more","more","vgood"]) #Set de valores que se convertirán al valor 3
    valor2 = set(["high","4","big","good"]) #Set de valores que se convertirán al valor 2
    valor1 = set(["med","3","acc"]) #Set de valores que se convertirán al valor 1
    valor0 = set(["low","2","small","unacc"]) #Set de valores que se convertirán al valor 0
    #Recorro toda la matriz y hago las conversiones correspondientes y lo guardo en la matriz creada
    for i in range(filas):
        for j in range(columnas):
            dato = matriz[i,j]
            if(dato in valor3):
                matriz_procesada[i,j] = 3
            elif(dato in valor2):
                matriz_procesada[i,j] = 2
            elif(dato in valor1):
                matriz_procesada[i,j] = 1
            elif(dato in valor0):
                matriz_procesada[i,j] = 0
    return matriz_procesada

def convertir_a_binario(numero, longitud): #Método para convertir un número en un vector con su valor binario
    vector_binario = np.zeros([longitud]) #Vector de las posiciones deseadas que guardará el valor en binario
    binario = bin(numero)[2:] #Método para obtener el valor binario de un número
    for i in range(longitud-len(binario)): #En caso que el valor en binario tenga longitud menor a  la deseada
        binario = '0' + binario   #Se le agregan ceros a la izquierda
#     print (etiqueta ," en binario es ", binario)
    for i in range(longitud):
        vector_binario[i] = binario[i] #Guardar el valor en binario en la posición correspondiente, cada posición del vector es un caracter del numero en binario
    return vector_binario.astype(int) #Devuelvo el vector como entero

# temp = convertir_a_binario(0,1)
# print(temp)

def etiquetas_a_binario(vector): #Método para convertir un vector de números a un vector de vectores con su valor binario
    filas = len(vector) #Numero de filas en el vector
    maximo = vector.max().astype(int) #Numero máximo del vector
    maximo_binario = bin(maximo)[2:] #Su equivalente en binario
    longitud = len(maximo_binario) #Su longitud, determina la longitud de cada posición del vector final
#     longitud = len(bin((vector.max()).astype(int))[2:]) #La longitud del valor en binario basado en el número mayor del vector, determina la longitud del vector final
    etiquetas_binarias = np.zeros((filas,longitud)) #Matriz que se llenará con las etiquetas convertidas a binario
    for i in range(filas): #Para cada fila en el vector
        etiqueta = vector[i] #Obtengo la etiqueta que quiero convertir a binario
        etiquetas_binarias[i] = convertir_a_binario(int(etiqueta),longitud) #La convierto en binario
    return etiquetas_binarias 

def matriz_a_binario(matriz): #Método para convertir una matriz de numeros a una matriz con sus valores en binario como vectores
    #     longitud = len(bin((vector.max()).astype(int))[2:]) #La longitud del valor en binario basado en el número mayor del vector, determina la longitud del vector final
    maximo = matriz.max().astype(int) #Valor máximo en la matriz
    maximo_binario = bin(maximo)[2:] #Su equivalente en binario
    longitud = len(maximo_binario) #Su longitud, determina la longitud de cada posición de la matriz final
    filas = np.shape(matriz)[0] 
    columnas = np.shape(matriz)[1]
    matriz_binaria = np.zeros((filas,columnas,longitud)) #Creo una matriz con las mismas dimensiones de matriz pero cada posición es un vector
    for i in range(filas):
        for j in range(columnas):
            dato = matriz[i,j]
            matriz_binaria[i,j] = convertir_a_binario(dato,longitud)
    return matriz_binaria

In [4]:
#Preprocesamos los datos y los convertimos a binario
datos = preprocesar(data)
# datos = matriz_a_binario(datos_preprocesados)

#los separamos en datos y etiquetas
X, etiquetas = datos[:,:-1], datos[:,-1]
print(X.shape)

etiquetas_binarias = etiquetas_a_binario(etiquetas)
#reordenamos las etiquetas para que queden en formato columna
y = np.vstack(etiquetas_binarias)
print(y)

(1728, 6)
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [1. 0.]
 [1. 1.]]


In [5]:
#Hacemos la función para la remoción de la media, nuestra idea es que la desviación estándar sea 1 y la media sea 0
def Remocion(X): #definimos la función
    X = X - X.mean(axis=0) # a los datos les restamos el valor de la media
    X = X/X.std(axis=0) #al resultado lo dividimos por la desviación estándar
    return X #retornamos los datos normalizamos
X_remocion = Remocion(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_remocion,y,test_size=0.3, random_state=5)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1209, 6) (1209, 2)
(519, 6) (519, 2)


In [7]:
def minimo_y_maximo(train,test):
    minimo = train.min()
    if minimo > test.min():
        minimo = test.min()
    maximo = train.max()
    if maximo < test.max():
        maximo = test.max()
    print(minimo,maximo)
    return minimo, maximo

In [8]:
#definimos los valores máximos y mínimos que puede tomar cada dimensión
dim1_min, dim1_max = minimo_y_maximo(X_train[0],X_test[0])
dim2_min, dim2_max = minimo_y_maximo(X_train[1],X_test[1])
dim3_min, dim3_max = minimo_y_maximo(X_train[2],X_test[2])
dim4_min, dim4_max = minimo_y_maximo(X_train[3],X_test[3])
dim5_min, dim5_max = minimo_y_maximo(X_train[4],X_test[4])
dim6_min, dim6_max = minimo_y_maximo(X_train[5],X_test[5])

#Número de neuronas en la capa de salida
num_output = y_train.shape[1]
#tenemos un conjunto de datos de dos dimensiones, definimos un perceptron
# con dos neuronas a la entrada y le asignamos una a cada dimensión
dim1 = [dim1_min, dim1_max]
dim2 = [dim2_min, dim2_max]
dim3 = [dim3_min, dim3_max]
dim4 = [dim4_min, dim4_max]
dim5 = [dim5_min, dim5_max]
dim6 = [dim6_min, dim6_max]

perceptron = nl.net.newp([dim1,dim2,dim3,dim4,dim5,dim6], num_output)

#entrenamos el perceptron con el conjunto de entrenamiento 
progreso_error = perceptron.train(X_train, y_train, epochs=2000, show=500, lr=0.3)
perceptron.layers[0].np['w'] #con esta instrucción conocemos los pesos para la capa de entrada

-1.3416407864998738 1.3416407864998738
-1.224744871391589 1.3416407864998738
-1.3416407864998738 1.3416407864998738
-1.336306209562106 1.3416407864998738
-1.3416407864998738 1.3416407864998738
-1.224744871391589 1.3416407864998738
Epoch: 500; Error: 157.0;
Epoch: 1000; Error: 157.0;
Epoch: 1500; Error: 157.0;
Epoch: 2000; Error: 157.0;
The maximum number of train epochs is reached


array([[-2.01246118, -2.28078934, -0.13416408,  1.20267559,  0.36742346,
         1.83711731],
       [ 0.40249224, -0.13416408, -0.40249224,  1.12249722,  0.36742346,
         1.10227038]])

In [10]:
y_pred = perceptron.sim(X_test)
# print('El acierto de clasificación es: ', (y_test==y_pred).sum()/len(y_test)*100, '%')
print('El acierto de clasificación es: ', (y_test==y_pred).sum()/(np.shape(y_test)[0]*np.shape(y_test)[1])*100, '%')
print((y_test == y_pred).sum(), y_pred.shape, (np.shape(y_test)[0]*np.shape(y_test)[1]))
print('El acierto de clasificación es: ', (y_test[0]==y_pred[0] & y_test[1]==y_pred[1]).sum()/len(y_test)*100, '%')

El acierto de clasificación es:  86.03082851637764 %
893 (519, 2) 1038


TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''